## Importing all the libraries

In [1]:
#Importing the libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install boto3

     |████████████████████████████████| 132 kB 22.4 MB/s eta 0:00:01
  Using cached s3transfer-0.5.1-py3-none-any.whl (79 kB)
     |████████████████████████████████| 8.5 MB 68.7 MB/s eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.18.14
    Uninstalling botocore-1.18.14:
      Successfully uninstalled botocore-1.18.14
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.3.3
    Uninstalling s3transfer-0.3.3:
      Successfully uninstalled s3transfer-0.3.3
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

awscli 1.18.155 requires botocore==1.18.14, but you'll have botocore 1.24.2 which is incompatible.
awscli 1.18.155 requires s3transfer<0.4.0,>=0.3.0, but you'll have s3transfer

In [3]:
!pip install s3fs

  Using cached s3fs-2022.1.0-py3-none-any.whl (25 kB)
Processing ./.cache/pip/wheels/c2/3f/42/08dc768873808d05f326167ead3fdc6f1b742a6253bd3ec6a3/aiobotocore-2.1.1-py3-none-any.whl
  Using cached aiohttp-3.8.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.3 MB)
  Using cached fsspec-2022.1.0-py3-none-any.whl (133 kB)
  Using cached botocore-1.23.24-py3-none-any.whl (8.4 MB)
  Using cached aioitertools-0.9.0-py3-none-any.whl (22 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached yarl-1.7.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (308 kB)
  Using cached multidict-6.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (121 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
  Using cached frozenlist-1.3.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_

## Processing 100K charts and creating the main data

In [4]:
import boto3

In [5]:
s3 = boto3.client('s3')

In [6]:
s3 = boto3.resource('s3')

In [7]:
# Reading medicine_mapping to create parent_rxcui_mapping
medicine_mapping = pd.read_csv("medicine_with_name_V1.0.csv")
medicine_mapping.head()

,rx,rxcui,parent_rxcui,parent_name
0,saw palmetto extract,236344,236344,saw palmetto extract
1,nivestym injectable product,2057204,68442,filgrastim
2,naprelan oral product,1178761,7258,naproxen
3,vistaril 25 mg oral capsule,995255,5553,hydroxyzine
4,latrodectus mactans antivenin 6000 unt,1867738,89887,Latrodectus mactans antivenin


In [8]:
def meat_text(meat):
    """
    Function to obtain a list of text for every code1 with meat which has the type as medicine

    Parameters
    ----------
    meat : entities["meat_entities"]
        Series/column of meat_entities for every code1

    Returns
    -------
    text : list
        Returns a list of text for every code1 with meat which has type as medicine
    """
    text = []
    for i in meat:
        if i["type"].lower() == 'medicine':
            text.append(i["text"].lower())
    return text

In [9]:
def rxcui_mapping(meat_text):
    """
    Function to obtain a list of parent_rxcui_mapping for every code1 with meat which has the type as medicine

    Parameters
    ----------
    meat_text : entities["meat_entities_text"]
        Series/column of meat_entities_text for every code1

    Returns
    -------
    parent_rxcui_mapping : list
        Returns a list of parent_rxcui_mapping for every code1 with meat which has the type as medicine
    """
    parent_rxcui_mapping = []
    for i in meat_text:
        if i in medicine_mapping.rx.values:
            parent_rxcui_mapping.append(medicine_mapping[medicine_mapping["rx"] == i]["parent_rxcui"].values)
    return parent_rxcui_mapping 

In [ ]:
import os
import json
import boto3
import io
import requests

from datetime import datetime
start_time = datetime.now()

bucket_name = 'epicypher'
folder_name = 'Experiments/Aishwarya/Autocoding/data/'
s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket(bucket_name)

charts_with_no_enc_id = []
charts_with_enc_id = []
data_with_enc_id = []
data_with_no_enc_id = []

number_of_files = 0

for obj in bucket.objects.filter(Prefix=folder_name):
    if number_of_files == 120000:
        break

    if number_of_files >= 0:
        
        print(number_of_files)
            
        i = str(obj.key).split('Experiments/Aishwarya/Autocoding/data/')[-1]

        s3_client = boto3.client('s3')
        json_obj = s3_client.get_object(Bucket=bucket_name, Key=obj.key)
        json_data = json_obj["Body"].read().decode('utf-8')
        data1 = json.loads(json_data)

        if 'entities_with_highlights' not in data1.keys():
            continue
        entities = pd.json_normalize(data1,record_path = ['entities_with_highlights'],meta=['NLP_VERSION'])

        if len(entities):

            if 'enc_id' not in entities.columns:
                charts_with_no_enc_id.append(i)
                
                entities['meat_entities'] = entities.apply(lambda x: [data1['meat_entities'][i] for i in x['meat']] if x['type']=='Disease' else None, axis=1)

                entities['meat_entities_text'] = entities["meat_entities"].map(lambda x: meat_text(x) if x else None)

                entities['meat_entities_type'] = entities.apply(lambda x: [data1['meat_entities'][i]["type"].lower() for i in x['meat']] if x['type']=='Disease' else None, axis=1)

                entities["parent_rxcui_mapping"] = entities["meat_entities_text"].map(lambda x: rxcui_mapping(x) if x else None)

                entities["final_code_family"] = entities["final_codes"].apply(lambda x: [i[:3] for i in x] if x else None)
                
                encounters = pd.json_normalize(data1,record_path = ['encounters'])
                
                entities["enc_id"] = entities["page"].apply(lambda x: encounters[(encounters['start'] <= x) & (encounters['end'] >= x)]["enc_id"].iloc[0])
             
                final_data = entities.merge(encounters,on ='enc_id')

                final_data['chart_id'] = i 

                data_with_no_enc_id.append(final_data)            
            else:
                charts_with_enc_id.append(i)

                entities['meat_entities'] = entities.apply(lambda x: [data1['meat_entities'][i] for i in x['meat']] if x['type']=='Disease' else None, axis=1)

                entities['meat_entities_text'] = entities["meat_entities"].map(lambda x: meat_text(x) if x else None)

                entities['meat_entities_type'] = entities.apply(lambda x: [data1['meat_entities'][i]["type"].lower() for i in x['meat']] if x['type']=='Disease' else None, axis=1)

                entities["parent_rxcui_mapping"] = entities["meat_entities_text"].map(lambda x: rxcui_mapping(x) if x else None)

                entities["final_code_family"] = entities["final_codes"].apply(lambda x: [i[:3] for i in x] if x else None)

                encounters = pd.json_normalize(data1,record_path = ['encounters'])

                final_data = entities.merge(encounters,on ='enc_id')

                final_data['chart_id'] = i 

                data_with_enc_id.append(final_data)
        
    number_of_files = number_of_files + 1
    
print("Number of charts with no enc_id -->",len(charts_with_no_enc_id))
print(" ")
print("Number of charts with enc_id -->",len(charts_with_enc_id))
print(" ")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [209]:
data_with_no_enc_id = pd.concat((x for x in data_with_no_enc_id),ignore_index = True)

In [210]:
data_with_enc_id = pd.concat((x for x in data_with_enc_id),ignore_index = True)

## Preparaing dataset for the model - 100k charts (Training data) and 20k charts (Test data)

In [53]:
data_with_no_enc_id_100k_105k = pd.read_csv("data_with_no_enc_id_100k_105k.csv")

In [55]:
data_with_no_enc_id_100k_105k.drop(columns = ['Unnamed: 0','x1', 'y1', 'x2', 'y2', 'h_page', 'w_page'],axis = 1,inplace = True)

#### We only need code1 with type as Disease as the training data

In [58]:
data_with_no_enc_id_100k_105k = data_with_no_enc_id_100k_105k[data_with_no_enc_id_100k_105k["type"] == "Disease"]

In [ ]:
data_with_no_enc_id_105k_110k = pd.read_csv("data_with_no_enc_id_105k_110k.csv")

In [62]:
data_with_no_enc_id_105k_110k.drop(columns = ['Unnamed: 0','x1', 'y1', 'x2', 'y2', 'h_page', 'w_page'],axis = 1,inplace = True)

In [65]:
data_with_no_enc_id_105k_110k = data_with_no_enc_id_105k_110k[data_with_no_enc_id_105k_110k["type"] == "Disease"]

In [ ]:
data_with_no_enc_id_110k_120k = pd.read_csv("data_with_no_enc_id_110k_120k.csv")

In [69]:
data_with_no_enc_id_110k_120k.drop(columns = ['Unnamed: 0','x1', 'y1', 'x2', 'y2', 'h_page', 'w_page'],axis = 1,inplace = True)

In [72]:
data_with_no_enc_id_110k_120k = data_with_no_enc_id_110k_120k[data_with_no_enc_id_110k_120k["type"] == "Disease"]

In [74]:
data_with_no_enc_id_100k_120k_merged = pd.concat([data_with_no_enc_id_100k_105k,data_with_no_enc_id_105k_110k,
                                                  data_with_no_enc_id_110k_120k],ignore_index=True)

In [75]:
data_with_no_enc_id_100k_120k_merged.shape

(1437454, 45)

In [85]:
data_with_no_enc_id_100k_120k_merged.to_csv("16thFeb2022_data_with_no_enc_id_100k_120k_main_data.csv")

In [ ]:
data_with_no_enc_id_100k_120k_merged = pd.read_csv("16thFeb2022_data_with_no_enc_id_100k_120k_main_data.csv")

In [5]:
data_with_no_enc_id_100k_120k_merged.drop(columns = ['Unnamed: 0','exact_match', 'additional_codes','checkbox'
                                         ,'start_x', 'end_x','start_y', 'end_y','HCC Description',
                                         'DX Description', 'DXCAT3DESC'],axis = 1,inplace = True)

In [ ]:
data_with_enc_id_100k_120k_merged = pd.read_csv("16thFeb2022_data_100k_120k_main_data.csv")

In [9]:
data_with_enc_id_100k_120k_merged.drop(columns = ['Unnamed: 0', 'additional_codes',
                                         'start_x', 'end_x','start_y', 'end_y','HCC Description',
                                         'DX Description', 'DXCAT3DESC'],axis = 1,inplace = True)

In [11]:
data_100k_120k_merged = pd.concat([data_with_enc_id_100k_120k_merged,data_with_no_enc_id_100k_120k_merged],ignore_index=True)

In [17]:
data_100k_120k_merged.to_csv("16thFeb2022_100k_120k_merged_main_data.csv")

## Reading data to merge with sf data and prepare the output variable

In [2]:
df = pd.read_csv("16thFeb2022_100k_120k_merged_main_data.csv")

In [3]:
import ast

In [4]:
df.type.value_counts()

Disease    1805764
Name: type, dtype: int64

In [5]:
df.columns

Index(['Unnamed: 0', 'code1', 'code2', 'combo_add_codes', 'final_codes',
       'query_source', 'DX Codetype', 'DX Category', 'DX Condition Type',
       'DXCAT3', 'enc_id', 'is_meat', 'meat', 'id_', 'text', 'type', 'page',
       'section', 'DX Context', 'NLP_VERSION', 'meat_entities',
       'meat_entities_text', 'meat_entities_type', 'parent_rxcui_mapping',
       'final_code_family', 'telehealth_encounter', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.name', 'npi.location',
       'npi.speciality', 'npi.type', 'chart_id'],
      dtype='object')

In [6]:
df.drop(columns = ['Unnamed: 0', 'code2','id_','type','npi.name', 'npi.location','npi.speciality','type',
                                         'combo_add_codes', 'is_meat'],axis = 1,inplace = True)

#### Creating features - Count of type of meat for every code1

In [8]:
df["medicine_count"] = df["meat_entities_type"].apply(lambda x: x.count('medicine') if x != "[]" else 0)
df["labtest_count"] = df["meat_entities_type"].apply(lambda x: x.count('labtest') if x != "[]" else 0)
df["embedded radiology/lab results_count"] = df["meat_entities_type"].apply(lambda x: x.count('embedded radiology/lab results') if x != "[]" else 0)
df["devices_count"] = df["meat_entities_type"].apply(lambda x: x.count('devices') if x != "[]" else 0)
df["abnormal findings_count"] = df["meat_entities_type"].apply(lambda x: x.count('abnormal findings') if x != "[]" else 0)
df["generic term_count"] = df["meat_entities_type"].apply(lambda x: x.count('generic term') if x != "[]" else 0)
df["plan_count"] = df["meat_entities_type"].apply(lambda x: x.count('plan') if x != "[]" else 0)
df["procedure_count"] = df["meat_entities_type"].apply(lambda x: x.count('procedure') if x != "[]" else 0)
df["radiology_count"] = df["meat_entities_type"].apply(lambda x: x.count('radiology') if x != "[]" else 0)
df["labs_count"] = df["meat_entities_type"].apply(lambda x: x.count('labs') if x != "[]" else 0)
df["rx_count"] = df["meat_entities_type"].apply(lambda x: x.count('rx') if x != "[]" else 0)
df["referral_count"] = df["meat_entities_type"].apply(lambda x: x.count('referral') if x != "[]" else 0)
df["symptoms_count"] = df["meat_entities_type"].apply(lambda x: x.count('symptoms') if x != "[]" else 0)
df["therapy_count"] = df["meat_entities_type"].apply(lambda x: x.count('therapy') if x != "[]" else 0)
df["procedures_count"] = df["meat_entities_type"].apply(lambda x: x.count('procedures') if x != "[]" else 0)
df["others_count"] = df["meat_entities_type"].apply(lambda x: x.count('others') if x != "[]" else 0)
df["physical_exam_count"] = df["meat_entities_type"].apply(lambda x: x.count('physical exam') if x != "[]" else 0)
df["evaluatory_term_count"] = df["meat_entities_type"].apply(lambda x: x.count('evaluatory term') if x != "[]" else 0)
df["imaging_count"] = df["meat_entities_type"].apply(lambda x: x.count('imaging') if x != "[]" else 0)

In [9]:
df.chart_id.value_counts()

39757998.json     3421
39784392.json     2958
39403257.json     2723
469388729.json    2622
39756115.json     2572
                  ... 
39747147.json        1
39393698.json        1
468783093.json       1
49236879.json        1
39747336.json        1
Name: chart_id, Length: 19961, dtype: int64

In [11]:
df["code1"].value_counts(normalize = True)*100

E119      15.040670
000       11.013067
Z684       8.395283
J449       4.061162
I10        3.567354
            ...    
C9101      0.000055
I63012     0.000055
M0608      0.000055
C6310      0.000055
D641       0.000055
Name: code1, Length: 3383, dtype: float64

In [12]:
df.drop(df[df.code1 == "000"].index,inplace=True)

In [13]:
df.drop_duplicates(keep = 'first',inplace = True)

In [14]:
df['DX Codetype'].value_counts()

Medicare Part C    1400150
Commercial           28647
commercial           16502
medicare_part_c      10152
TNM                   1667
Name: DX Codetype, dtype: int64

In [15]:
df["code1"].value_counts()

E119     250298
Z684     123777
J449      67896
I10       61087
I509      34586
          ...  
F1098         1
M9050         1
C9240         1
C8416         1
I519          1
Name: code1, Length: 3382, dtype: int64

In [16]:
df.parent_rxcui_mapping.value_counts()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          4903
[array([6809])]                                                                                                                                                                                                                                                                                                        

#### Cleaning parent_rxcui_mapping and chart_id

In [17]:
df['parent_rxcui_mapping'] = df['parent_rxcui_mapping'].str.replace('array(())','')

In [18]:
df["parent_rxcui_mapping"] = df["parent_rxcui_mapping"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [19]:
df["parent_rxcui_mapping"] = df["parent_rxcui_mapping"].apply(lambda x:[j for i in x for j in i] if type(x) != float and x != None else None)

In [20]:
df["parent_rxcui_mapping"]

0                                                     [6809]
1                                                     [6809]
2          [29046, 29046, 29046, 29046, 29046, 29046, 29046]
3                                                     [6809]
4                                                    [17767]
                                 ...                        
1805759                                                   []
1805760                                                   []
1805761                                                   []
1805762                                                   []
1805763                                                   []
Name: parent_rxcui_mapping, Length: 1457118, dtype: object

In [21]:
df['chart_id'] = df['chart_id'].str.replace('.json','')

In [23]:
df.to_csv("16thFeb2022_100_120_merged_main_data.csv")

In [24]:
df2 = pd.read_csv("16thFeb2022_100_120_merged_main_data.csv")

In [25]:
df2.columns

Index(['Unnamed: 0', 'code1', 'final_codes', 'query_source', 'DX Codetype',
       'DX Category', 'DX Condition Type', 'DXCAT3', 'enc_id', 'meat', 'text',
       'page', 'section', 'DX Context', 'NLP_VERSION', 'meat_entities',
       'meat_entities_text', 'meat_entities_type', 'parent_rxcui_mapping',
       'final_code_family', 'telehealth_encounter', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.type', 'chart_id',
       'medicine_count', 'labtest_count',
       'embedded radiology/lab results_count', 'devices_count',
       'abnormal findings_count', 'generic term_count', 'plan_count',
       'procedure_count', 'radiology_count', 'labs_count', 'rx_count',
       'referral_count', 'symptoms_count', 'therapy_count', 'procedures_count',
       'others_count', 'physical_exam_count', 'evaluatory_term_count',
       'imaging_count'],
      dtype='object')

In [26]:
df2.drop(columns = ['Unnamed: 0','meat_entities', 'meat_entities_text','meat_entities_type'],axis = 1, inplace = True)

In [27]:
df2.drop_duplicates(keep = 'first',inplace = True)

## Salesforce's Data

In [28]:
sf = pd.read_csv("sf_w_comments.csv")

In [29]:
sf.head()

,project_id,chart_id,code,l1_finish_time,l1_pc
0,2001210431,Cambia_MRA_4RD9MD6ND13_100000000153_910433740_...,G40209,2021-12-14 08:19:30.000,Add
1,2001210431,Cambia_MRA_4RD9MD6ND13_100000000153_910433740_...,NaN,2021-12-14 08:19:30.000,NaN
2,2001210431,Cambia_MRA_4RD9MD6ND13_100000000153_910433740_...,D580,2021-12-14 08:19:30.000,ASR
3,2001210431,Cambia_MRA_4RD9MD6ND13_100000000153_910433740_...,E039,2021-12-14 08:19:30.000,Add
4,2001210431,Cambia_MRA_4RD9MD6ND13_100000000153_910433740_...,E039,2021-12-14 08:19:30.000,ASR


In [30]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17650238 entries, 0 to 17650237
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   project_id      object
 1   chart_id        object
 2   code            object
 3   l1_finish_time  object
 4   l1_pc           object
dtypes: object(5)
memory usage: 673.3+ MB


In [31]:
sf2 = sf.drop_duplicates(keep = 'first')

In [32]:
df2["chart_id"] = df2["chart_id"].astype(str)

In [33]:
df2["code1"] = df2["code1"].astype(str)

In [34]:
sf2["chart_id"] = sf2["chart_id"].astype(str)

In [35]:
sf2["code"] = sf2["code"].astype(str)

In [36]:
final = df2.merge(sf2, left_on=['chart_id','code1'], right_on = ['chart_id','code'], how='left')
final.head()

,code1,final_codes,query_source,DX Codetype,DX Category,DX Condition Type,DXCAT3,enc_id,meat,text,...,therapy_count,procedures_count,others_count,physical_exam_count,evaluatory_term_count,imaging_count,project_id,code,l1_finish_time,l1_pc
0,E119,['E119'],exact,Medicare Part C,19.0,Chronic,E11,1,"['3-526', '5-1460', '5-1479']",Diabetes mellitus type 2,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,E119,['E119'],exact,Medicare Part C,19.0,Chronic,E11,1,"['3-526', '5-1460', '5-1479']",Diabetes,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,I219,['I219'],exact,Medicare Part C,86.0,Acute,I21,1,"['3-1949', '5-1171', '5-1202', '6-389', '6-799...",heart attack,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,E119,['E119'],exact,Medicare Part C,19.0,Chronic,E11,1,"['3-526', '5-1460', '5-1479']",DMT2,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,I4891,['I4891'],exact,Medicare Part C,96.0,Acute,I48,2,['8-155'],atrial fibrillation,...,0,0,0,0,0,0,2001211088,I4891,2021-10-25 10:09:28.000,Add


In [38]:
import numpy as np

#### Creating the output variable 

In [39]:
final["output"] = np.where(final["code1"] == final["code"],1,0)

In [40]:
final.columns

Index(['code1', 'final_codes', 'query_source', 'DX Codetype', 'DX Category',
       'DX Condition Type', 'DXCAT3', 'enc_id', 'meat', 'text', 'page',
       'section', 'DX Context', 'NLP_VERSION', 'parent_rxcui_mapping',
       'final_code_family', 'telehealth_encounter', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.type', 'chart_id',
       'medicine_count', 'labtest_count',
       'embedded radiology/lab results_count', 'devices_count',
       'abnormal findings_count', 'generic term_count', 'plan_count',
       'procedure_count', 'radiology_count', 'labs_count', 'rx_count',
       'referral_count', 'symptoms_count', 'therapy_count', 'procedures_count',
       'others_count', 'physical_exam_count', 'evaluatory_term_count',
       'imaging_count', 'project_id', 'code', 'l1_finish_time', 'l1_pc',
       'output'],
      dtype='object')

In [41]:
#final.to_csv("16thFeb2022_100k_120k_data_merged_with_sf_v2.csv")